In [ ]:
print('Data Cleansing - Port distance')

In [2]:
import pandas.io.sql as sqlio
import psycopg2 as ps
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date, time, datetime
import math
import uuid

In [3]:
load_dotenv()
db = os.getenv('POSTGRES_DB')
user = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD')
host = os.getenv('POSTGRES_SERVER')
port = os.getenv('POSTGRES_PORT')

In [4]:
conn = ps.connect(dbname=db, 
                  user=user,
                  password=password, 
                  host=host,
                  port=port)

In [5]:
sql = """select * from stg_pelni.master_port mp
where mp.flag_ppss ='Y' and mp.is_deleted =false"""

In [6]:
# Read Query
df_sql = sqlio.read_sql_query(sql, conn)
df_sql2 = sqlio.read_sql_query(sql, conn)

C:\Users\BN001602001\AppData\Local\Temp\ipykernel_6612\3537067167.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sql = sqlio.read_sql_query(sql, conn)
C:\Users\BN001602001\AppData\Local\Temp\ipykernel_6612\3537067167.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sql2 = sqlio.read_sql_query(sql, conn)


In [8]:
df_sql = df_sql.loc[:, ['code', 'port_name']]
df_sql2 = df_sql2.loc[:, ['code', 'port_name']]

df_sql = df_sql.rename(columns={'code' : 'code_origin', 'port_name' : 'port_origin'})
df_sql2 = df_sql2.rename(columns={'code' : 'code_destination', 'port_name' : 'port_destination'})

In [9]:
# Read CSV
df = pd.read_excel('data/DataCleansing_Port_Distance.xlsx')
df.head()

,No,Pelabuhan Asal,Pelabuhan Tujuan,Nautis,Komersial
0,1,Tanjung Priok,Pulau Batam,521,NaN
1,2,Pulau Batam,Tanjung Balai Karimun,45,NaN
2,3,Tanjung Balai Karimun,Belawan - Medan,344,NaN
3,4,Tanjung Priok,Belawan - Medan,783,NaN
4,5,Tanjung Priok,Tanjung Balai Karimun,560,NaN


In [10]:
df['Komersial'] = df['Nautis']
df = df.rename(columns={'Pelabuhan Asal' : 'port_origin', 'Pelabuhan Tujuan' : 'port_destination', 'Nautis' : 'jarak_nautis', 'Komersial' : 'jarak_komersial'})
df

,No,port_origin,port_destination,jarak_nautis,jarak_komersial
0,1,Tanjung Priok,Pulau Batam,521,521
1,2,Pulau Batam,Tanjung Balai Karimun,45,45
2,3,Tanjung Balai Karimun,Belawan - Medan,344,344
3,4,Tanjung Priok,Belawan - Medan,783,783
4,5,Tanjung Priok,Tanjung Balai Karimun,560,560
...,...,...,...,...,...
181,182,Wanci,Kendari,124,124
182,183,Kendari,Bau-Bau,110,110
183,184,Sunda Kelapa,Tidung,29,29
184,185,Sorong,Waisai,37,37


In [11]:
df.loc[df["port_origin"] == "Babang [Bacan]", 'port_origin'] = "Babang"
df.loc[df["port_origin"] == "Benoa / Denpasar", 'port_origin'] = "Benoa"
df.loc[df["port_origin"] == "Ampenan / Lembar", 'port_origin'] = "Lembar"
df.loc[df["port_origin"] == "Teluk Bayur Padang", 'port_origin'] = "Teluk Bayur"
df.loc[df["port_origin"] == "Gunung Sitoli / Nias", 'port_origin'] = "Gunung Sitoli"
df.loc[df["port_origin"] == "Pulau Togian", 'port_origin'] = "Wakai, Togian"
df.loc[df["port_origin"] == "Karimun Jawa", 'port_origin'] = "Legon Bajak, Karimun Jawa"


df.loc[df["port_destination"] == "Babang [Bacan]", 'port_destination'] = "Babang"
df.loc[df["port_destination"] == "Benoa / Denpasar", 'port_destination'] = "Benoa"
df.loc[df["port_destination"] == "Ampenan / Lembar", 'port_destination'] = "Lembar"
df.loc[df["port_destination"] == "Teluk Bayur Padang", 'port_destination'] = "Teluk Bayur"
df.loc[df["port_destination"] == "Gunung Sitoli / Nias", 'port_destination'] = "Gunung Sitoli"
df.loc[df["port_destination"] == "Belawan - Medan", 'port_destination'] = "Belawan"
df.loc[df["port_destination"] == "Pulau Togian", 'port_destination'] = "Wakai, Togian"
df.loc[df["port_destination"] == "Karimun Jawa", 'port_destination'] = "Legon Bajak, Karimun Jawa"
df.loc[df["port_destination"] == "Pulau Batam", 'port_destination'] = "Batam/Batu Ampar"

In [12]:
df

,No,port_origin,port_destination,jarak_nautis,jarak_komersial
0,1,Tanjung Priok,Batam/Batu Ampar,521,521
1,2,Pulau Batam,Tanjung Balai Karimun,45,45
2,3,Tanjung Balai Karimun,Belawan,344,344
3,4,Tanjung Priok,Belawan,783,783
4,5,Tanjung Priok,Tanjung Balai Karimun,560,560
...,...,...,...,...,...
181,182,Wanci,Kendari,124,124
182,183,Kendari,Bau-Bau,110,110
183,184,Sunda Kelapa,Tidung,29,29
184,185,Sorong,Waisai,37,37


In [13]:
# Change Typedata
df['port_origin'] = df['port_origin'].astype(str)
df_sql['port_origin'] = df_sql['port_origin'].astype(str)

# Inner Join
merge_data = pd.merge(df_sql, 
                      df, 
                      on ='port_origin', 
                      how ='inner')

In [14]:
merge_data

,code_origin,port_origin,No,port_destination,jarak_nautis,jarak_komersial
0,662,Labuan Bajo,88,Bima,83,83
1,662,Labuan Bajo,120,Larantuka,220,220
2,662,Labuan Bajo,153,Pulau Rinca,75,75
3,662,Labuan Bajo,158,Makassar,210,210
4,662,Labuan Bajo,167,Marapokot,110,110
...,...,...,...,...,...,...
117,255,Serasan,63,Tambelan,136,136
118,255,Serasan,165,Pontianak,181,181
119,942,Ternate,48,Sorong,306,306
120,942,Ternate,106,Babang,153,153


In [15]:
df_sql2

,code_destination,port_destination
0,590,Bawean
1,662,Labuan Bajo
2,920,Amurang
3,952,Amahai
4,845,Karatung
...,...,...
98,725,Selat Lampa
99,942,Ternate
100,983,Pomako
101,251,Sei Kolak Kijang


In [16]:
# Change Typedata
df['port_destination'] = df['port_destination'].astype(str)
df_sql2['port_destination'] = df_sql2['port_destination'].astype(str)

# Inner Join
cleandf = pd.merge(df_sql2, 
                      merge_data, 
                      on ='port_destination', 
                      how ='inner')

In [ ]:
cleandf

In [17]:
for name in cleandf['port_origin'].unique():
    cleandf.loc[cleandf['port_origin'] == name, 'id_port_distance'] = uuid.uuid4()

C:\Users\BN001602001\AppData\Local\Temp\ipykernel_6612\2425716301.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '970bf31e-d89a-48d3-827f-4e7ecdb2a96b' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  cleandf.loc[cleandf['port_origin'] == name, 'id_port_distance'] = uuid.uuid4()


In [18]:
cleandf = cleandf.loc[:,['id_port_distance','port_origin', 'port_destination', 'code_origin', 'code_destination', 'jarak_nautis', 'jarak_komersial']]
cleandf

,id_port_distance,port_origin,port_destination,code_origin,code_destination,jarak_nautis,jarak_komersial
0,970bf31e-d89a-48d3-827f-4e7ecdb2a96b,Waingapu,Labuan Bajo,685,662,104,104
1,ff78a685-09d8-48e6-b8c3-775343778311,Ambon,Amahai,946,952,75,75
2,f2486bf0-649c-4850-8c5f-e4f63e73a6fc,Lirung,Karatung,844,845,65,65
3,169c979e-f303-46a7-9015-39b2a28ee43c,Tanjung Priok,Belinyu,431,322,325,325
4,3adeea82-d11b-40d6-9419-d204916a2e64,Sanana,Namlea,928,951,110,110
...,...,...,...,...,...,...,...
82,35ffb856-31b1-4c29-bbf3-25ec87875ef9,Ternate,Sorong,942,971,306,306
83,bacbac96-3ffd-4c94-a056-4b5d6906c8e7,Midai,Serasan,246,255,95,95
84,42ca7bef-a366-4bb1-ac3b-e90eee2a0286,Agats,Merauke,948,979,380,380
85,ff78a685-09d8-48e6-b8c3-775343778311,Ambon,Ternate,946,942,323,323
